In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


df=pd.read_excel('faltas_treinamento_sujo.xlsx')
df

In [ ]:
#Exercício 1 – Higienização Básica
#a) Verifique valores nulos e tipos incorretos.
df.isnull().sum()
#b) Corrija a coluna de datas inválidas.
df['Data_Ultima_Falta'] = pd.to_datetime(df['Data_Ultima_Falta'], errors='coerce', dayfirst=True)

#c) Normalize os nomes dos turnos e dias da semana.
Turno= {
    'manhã': 'Manhã',
    'tarde': 'Tarde',
    'noite': 'Noite',
    'madrugada': 'Madrugada',
    'manha': 'Manhã',
    'tarde': 'Tarde'
}
df['Turno'] = df['Turno'].str.lower().map(Turno).fillna(df['Turno'])
Dia_da_Semana_Ultima_Falta	= {
    'segunda': 'Segunda',
    'terca': 'Terça',
    'terça': 'Terça',
    'quarta': 'Quarta',
    'quinta': 'Quinta',
    'sexta': 'Sexta',
    'sex': 'Sexta',
    'sabado': 'Sábado',
    'domingo': 'Domingo'
}
df['Dia_da_Semana_Ultima_Falta'] = df['Dia_da_Semana_Ultima_Falta'].str.lower().map(Dia_da_Semana_Ultima_Falta).fillna(df['Dia_da_Semana_Ultima_Falta'])
df

In [ ]:
#Exercício 2 – Remoção e Substituição
#a) Remova os registros duplicados.
df.drop_duplicates()
#b) Substitua os valores nulos da coluna Evadiu por “Desconhecido”.
df['Evadiu'] = df['Evadiu'].fillna('Desconhecido')
#c) Substitua os valores nulos da coluna Data_Ultima_Falta pela data mais comum.
data_comum = df['Data_Ultima_Falta'].mode()[0]
df['Data_Ultima_Falta'] = df['Data_Ultima_Falta'].fillna(data_comum)
df

In [ ]:
#Exercício 3 – Correções e Detecção de Outliers
#a) Converta a coluna Faltas para tipo inteiro.
df['Faltas']=df['Faltas'].astype(int)
#b) Substitua outliers de faltas por np.nan e depois pela mediana.
Q1, Q3 = df['Faltas'].quantile([0.25, 0.75])
IQR = Q3 - Q1
limite_inferior, limite_superior = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
df['Faltas'] = df['Faltas'].apply(lambda x: np.nan if x < limite_inferior or x > limite_superior else x)
df['Faltas'] = df['Faltas'].fillna(df['Faltas'].median())
#c) Crie um flag “Faltas_Suspeitas” para valores > 10.
df['Faltas_suspeitas'] = df['Faltas'].apply(lambda x: True if x > 10 else False)
df

In [ ]:
#Exercício 4 – Análise Descritiva Avançada
#a) Qual o número total de evasões por departamento?
evasoes_por_departamento = df[df['Evadiu'] == 'Sim'].groupby('Departamento').size()

#b) Qual é o percentual de evasão considerando apenas os que participaram do turno da tarde?
turnotarde= df[df['Turno']=='Tarde']
evadiutarde=turnotarde[turnotarde['Evadiu']=='Sim']
qtstarde=len(evadiutarde)
total=len(turnotarde)
porcent=(qtstarde / total) * 100
print(f" {porcent:.2f}%")
#c) Qual é o desvio padrão de faltas por turno?
desviopadrao = df.groupby('Turno')['Faltas'].std().round(2).astype(str) + '%'
desviopadrao

In [ ]:
#Exercício 5 – Análise Temporal
#a) Em qual mês ocorreu o maior número de faltas?
df['Mês'] =df['Data_Ultima_Falta'].dt.month
faltasmes = df.groupby('Mês')['Faltas'].sum()
mesmaisfaltas = faltasmes.idxmax()
faltas_no_mes = faltasmes.max()
print(f'mes com mais falta: {mesmaisfaltas}, quantidade de faltas :{faltas_no_mes}')
#b) Crie uma coluna Semana_Ultima_Falta e descubra qual semana teve maior concentração de ausências.
df['Data_Ultima_Falta'] = pd.to_datetime(df['Data_Ultima_Falta'])
df['Semana_Ultima_Falta'] = df['Data_Ultima_Falta'].dt.isocalendar().week
faltas_por_semana = df.groupby('Semana_Ultima_Falta')['Faltas'].count()
semana_mais_faltas = faltas_por_semana.idxmax()
total_faltas = faltas_por_semana.max()
print(f'A semana {semana_mais_faltas} teve a maior concentração de faltas, com {total_faltas} ausências.')
#c) Faça uma análise cruzando dia da semana e quantidade de evasões.
faltas_por_dia = df.groupby('Dia_da_Semana_Ultima_Falta')['Faltas'].count()
dia_mais_faltas = faltas_por_dia.idxmax()
concentracao_maxima = faltas_por_dia.max()
print(f'O dia da semana com mais faltas foi {dia_mais_faltas} com {concentracao_maxima} faltas.')



In [ ]:
#Exercício 6 – Análise de Risco
#a) Crie a coluna Risco_Evasao com três categorias: alto,moderado e baixo;.
df['Risco_Evasao'] = pd.cut(df['Faltas'], bins=[-1, 2, 4, 6], labels=['Baixo', 'Moderado', 'Alto'])
#b) Gere uma tabela com a distribuição percentual de risco por departamento.
tabela = pd.crosstab(df['Departamento'], df['Risco_Evasao'])
percentual = tabela.div(tabela.sum(axis=1), axis=0) * 100
print(percentual.round(2))
#c) Há relação entre risco alto e turno da tarde?
risco_alto = df[df['Risco_Evasao'] == 'Alto']
contagem_alto_por_turno = risco_alto['Turno'].value_counts()
total_por_turno = df['Turno'].value_counts()
proporcao_alto_por_turno = (contagem_alto_por_turno / total_por_turno) * 100
print(proporcao_alto_por_turno.round(2))


In [ ]:
#Exercício 7 – Visualizações Estratégicas
#a) Gráfico de barras: número de faltas por departamento.
faltas_por_departamento = df.groupby('Departamento')['Faltas'].sum()
plt.figure(figsize=(10, 6))
sns.barplot(x=faltas_por_departamento.index, y=faltas_por_departamento.values, palette='Blues_d')
plt.title('Número de Faltas por Departamento')
plt.xlabel('Departamento')
plt.ylabel('Número de Faltas')
plt.xticks(rotation=45) 
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()  
plt.show()

In [ ]:
#b) Gráfico de pizza: distribuição de risco de evasão.



distribuicao_risco = df['Risco_Evasao'].value_counts()


plt.figure(figsize=(10, 6)) 
plt.pie(distribuicao_risco, labels=distribuicao_risco.index, autopct='%1.1f%%', startangle=90, colors = ['#FF5733', '#FFC300', '#DAF7A6']  
 )

plt.title('Distribuição de Risco de Evasão', )
plt.axis('equal')  
plt.show()



In [ ]:
#c) Heatmap cruzando Turno e Risco_Evasao.
tabelac = pd.crosstab(df['Turno'], df['Risco_Evasao'])
plt.figure(figsize=(8, 6)) 
sns.heatmap(tabelac, annot=True, fmt='d', 	cmap='OrRd', cbar=True, linewidths=0.5)
plt.title('Heatmap: Turno vs Risco de Evasão')
plt.xlabel('Risco de Evasão')
plt.ylabel('Turno')
plt.tight_layout()
plt.show()


ANALISE


1: Turno com mais faltas: Manhã

2: Semana com mais faltas: semana 12 teve a maior concentração de faltas, com 33 ausências

3: Departamento com mais faltas: Operações

4:Turno com maior risco de evasão : Manhã 

5:Turno com menor risco de evasão : Tarde 

6: Dia com mais faltas: Segunda com 78 faltas

7: Risco de evasão dos alunos: Baixo

8: Departamento com menos faltas: TI